# DNA extraction of  96-well cultures with magnetic beads

## Protocol Initialization
Ensure you have calibrated the pipettes before doing this protocol

In [1]:
from opentrons import execute, simulate
import json
import time

Use one of the code lines below for simulation or execution mode:

**Simulation mode**
```
protocol = simulate.get_protocol_api('2.13')
```



**Execution mode**
```
protocol = execute.get_protocol_api('2.13')
```

In [2]:
protocol = simulate.get_protocol_api('2.12')

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [3]:
protocol.home()

Out of bounds move: X=(418.00000000000006 motor controller, 418.54571439077444 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 418.54571439077444 deck) too high for limit 418.0


## Declaring Labware
- **Right mount**: An 8-channel p300 pipette
- **Slot 1**: 12-well reservoir with buffers
    - Well 1: Lysis buffer
    - Well 2: Neutralization buffer
    - Well 3: Wash buffer
    - Well 4: Ethanol
    - Well 5: Elution Buffer
- **Slot 4**: 96-deepwell plate (square wells) with ressuspended bacterial cell pellets
- **Slot 7**: Magnetic module with 96-deepwell plate (square wells) with 100uL of magnetic beads
- **Slot 9**: Output plate in 96-well format
- **Slot 10**: 1-well reservoir for buffer waste
- **Slots 2, 3**: Tipracks for p300

Replace with appropriate labware codes in the cell below if needed
If you need to use custom labware, use our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [4]:
# Set labware
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    plate_96 = protocol.load_labware_from_definition(labware_def, 4)

plate_out = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '9')

waste = protocol.load_labware('nest_1_reservoir_195ml', '4')

with open ('custom_labware/enzymax_12_reservoir_20ml.json') as labware_file:
    labware_def = json.load(labware_file)
    liquids_reservoir = protocol.load_labware_from_definition(labware_def, 1)
    
tipracks_300 = [protocol.load_labware('opentrons_96_tiprack_300ul', '2'),
               protocol.load_labware('opentrons_96_tiprack_300ul', '3')]

p300 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=tipracks_300)

mag_module = protocol.load_module("magnetic module", '7')
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    mag_plate = mag_module.load_labware_from_definition(labware_def, 7)

# Define the number of samples here
- Run the next cell and fill out the prompts
- You can use from 1 to 96 samples.

In [6]:
sample_no = 16


col_no = 1+(sample_no-1)//8
columns = plate_96.rows()[0][:col_no]
lysis_buff = liquids_reservoir['A1']
neut_buff = liquids_reservoir['A2']
wash_buff = liquids_reservoir['A3']
ethanol = liquids_reservoir['A4']
elut_buff = liquids_reservoir['A5']

## Labware Calibration

Change the x, y, and z variables (**in mm**) in the "set_offset" functions and run each cell repeatedly until the positioning is correct for that piece of labware. You will have to use a "dummy" plate to calibrate the labware that is not on the deck yet (competent cells).

### Tipracks:

In [7]:
protocol.comment('-- CALIBRATION --')

In [8]:
# Calibrate tiprack for p300. Re-run calibration cell with different numbers if needed
tipracks_300[0].set_offset(x=0.7, y=0.4, z=0.)
p300.move_to(tipracks_300[0]['A1'].top())

Out of bounds move: X=(418.00000000000006 motor controller, 418.54571439077444 deck) too high for limit 418.0


<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [9]:
# Calibrate tiprack for p300. Re-run calibration cell with different numbers if needed
tipracks_300[1].set_offset(x=0.7, y=0.6, z=0.)
p300.move_to(tipracks_300[1]['A1'].top())

<InstrumentContext: p300_multi_v2.1 in RIGHT>

### Plates/Racks:

In [10]:
p300.pick_up_tip()

<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [11]:
# Calibrate remaining labware. Re-run calibration cells with different numbers if needed
plate_96.set_offset(x=0.0, y=0.5, z=0.7)
p300.move_to(plate_96['A1'].top())

<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [12]:
liquids_reservoir.set_offset(x=0.0, y=0.0, z=0.0)
p300.move_to(liquids_reservoir['A1'].top())

<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [14]:
mag_plate.set_offset(x=0.0, y=0.9, z=0.7)
p300.move_to(mag_plate['A1'].top())

<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [16]:
plate_out.set_offset(x=0.5, y=0.5, z=0.3)
p300.move_to(plate_out['A1'].top())

<InstrumentContext: p300_multi_v2.1 in RIGHT>

In [ ]:
waste.set_offset(x=0.0, y=0.0, z=0.0)
p300.move_to(waste['A1'].top())

In [17]:
p300.return_tip()
p300.reset_tipracks()

## Protocol Execution
### Add lysis and neutralization buffer
- Centrifuge at max speed for 20 minutes when done

In [18]:

times = []
protocol.comment('-- ADDING LYSIS BUFFER --')
p300.pick_up_tip()
for col in columns:
    p300.aspirate(120, lysis_buff)
    p300.dispense(120, col.top()))
    p300.blow_out()
    times.append({'column':col, 'time':time.time()})
p300.drop_tip()

remaining = 300 - (time.time() - times[0]['time'])
print(remaining, 'seconds wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(seconds = remaining)

protocol.comment('-- ADDING REMAINING NEUTRALIZATION BUFFER --')
p300.pick_up_tip()
for remain in times:
    col = remain['column']
    p300.aspirate(240, neut_buff)
    p300.dispense(240, col.top())
    p300.blow_out()
p300.drop_tip()

279.3387403488159 seconds wait started at 14:23:46


<InstrumentContext: p300_multi_v2.1 in RIGHT>

### Add supernatant and ethanol to beads
- Aspirate slowly and a bit higher to avoid cell debris
- After this step, incubate at room temperature for 20 minutes, vortexing briefly after every 4 minutes 

In [19]:
protocol.comment('-- ADDING SAMPLES AND EHTANOL TO MAG PLATE --')

p300.pick_up_tip()
for col in columns:
    p300.aspirate(200, ethanol)
    p300.dispense(200, mag_plate[col.well_name].top())
    p300.blow_out()
p300.drop_tip()

for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, col.bottom(z=3.0), rate=0.2)
    p300.dispense(300, mag_plate[col.well_name])
    p300.mix(5, 250, mag_plate[col.well_name], rate=3.0)
    p300.drop_tip()

### Remove supernatant from beads

In [20]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes = 2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, mag_plate[col.well_name], rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.aspirate(300, mag_plate[col.well_name], rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.drop_tip()
mag_module.disengage()

2 minutes wait started at 16:02:10


### Add wash buffer
- After this step, incubate at room temperature for 10 minutes, vortexing briefly after every 2 minutes

In [21]:
p300.pick_up_tip()
for col in columns:
    p300.aspirate(300, wash_buff)
    p300.dispense(300, mag_plate[col.well_name].top())
    p300.blow_out()
p300.drop_tip()

<InstrumentContext: p300_multi_v2.1 in RIGHT>

### Remove wash buffer
- After this step, dry plate at 65°C for 20 minutes

In [22]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes = 2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, mag_plate[col.well_name], rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.drop_tip()
mag_module.disengage()

2 minutes wait started at 16:07:12


### Add elution buffer
- After this step, incubate at room temperature for 10 minutes, vortexing briefly every 2 minutes

In [23]:
p300.pick_up_tip()
for col in columns:
    p300.aspirate(100, elut_buff)
    p300.dispense(100, mag_plate[col.well_name].top())
    p300.blow_out()
p300.drop_tip()

<InstrumentContext: p300_multi_v2.1 in RIGHT>

### Transfer elution to output plate

In [24]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes=2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(100, mag_plate[col.well_name], rate=0.2)
    p300.dispense(100, plate_out[col.well_name])
    p300.drop_tip()

mag_module.disengage()

2 minutes wait started at 16:11:38


In [25]:
protocol.home()

Out of bounds move: X=(418.00000000000006 motor controller, 418.54571439077444 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 418.54571439077444 deck) too high for limit 418.0


In [ ]:
protocol.commands()